In [45]:
!pip3 install ckiptagger
!pip3 install tensorflow-gpu
!pip3 install gdown
!pip install -U ckiptagger[tf,gdown]
!pip3 install jieba


'pip3' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'pip3' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'pip3' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'pip' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'pip3' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'pip3' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [47]:
pip install ckiptagger

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import time
import os

In [2]:
df = pd.read_csv('juicy_bun_comment.csv')
df

,time,name,commenter,comment,star
0,8 個月前,浩宇許,在地嚮導 · 20 則評論,餐點便宜大份，味道也很好很有美式的感覺。今天吃的是牛雞漢堡，紐澳良烤雞腿和厚實的牛肉漢堡排，...,5
1,1 個月前,Agnes Li,在地嚮導 · 23 則評論,學生家庭最愛飽足CP店，佛系不收服務費，飲料無限喝，熱飲也只要+40喝好喝滿心靈大滿足。\n...,5
2,9 個月前,Nonie Chung,在地嚮導 · 30 則評論,服務人員很親切 東西很乾淨 薯條顏色很漂亮 ，室內沒有油煙，貝果很驚豔 大推 ！,5
3,9 個月前,Shih Wang,在地嚮導 · 52 則評論,*希臘野菇蛋捲\n歐姆蛋很好吃！\n*戰骨野菇蛋捲\n有很大塊的豬排， 味道很香很好吃，CP...,5
4,7 個月前,林旻諭,在地嚮導 · 62 則評論,花生醬給的很大方，但有點太膩了\n歐姆蛋比較合我胃口，蛋裡還包有馬鈴薯之類的，可以增加飽足感😍,4
...,...,...,...,...,...
185,1 年前,Jeremy Chang,在地嚮導 · 55 則評論,CP值高，份量十足。\n招牌起司漢堡裡面有附洋芋片，蠻特別的配置。薯條可以加價變大份的！如果...,5
186,2 個月前,許薰,13 則評論,好吃 店員親切 cp超高 回訪三次了,5
187,3 年前,A Alice,1 則評論,很美式的餐廳🍴 價格又親民\n跟網路說的一樣好評 下次來動物園玩 也還是要來吃‼️,5
188,1 週前,Marcus Cho,6 則評論,東西好吃，闆娘又美\n推推,5


In [4]:
dict = list(df.comment.map(lambda x:re.sub('[^\w]', '', str(x)))) #去除特殊符号（如：#、\n等）
dict

['餐點便宜大份味道也很好很有美式的感覺今天吃的是牛雞漢堡紐澳良烤雞腿和厚實的牛肉漢堡排配上醬汁和兩種起司搭配好吃飲料可以續杯店員都有一直留意且主動詢問是否需要續杯',
 '學生家庭最愛飽足CP店佛系不收服務費飲料無限喝熱飲也只要40喝好喝滿心靈大滿足店員都很年輕是政大商圈一定要推的社區長壽店好店點想要求用餐環境優雅坐位舒服的建議去對面Bistro會比較合適這邊是吃飽喝足喧嘩聊天的偏美式隨性空間十年居民再訪幾十次加分推薦',
 '服務人員很親切東西很乾淨薯條顏色很漂亮室內沒有油煙貝果很驚豔大推',
 '希臘野菇蛋捲歐姆蛋很好吃戰骨野菇蛋捲有很大塊的豬排味道很香很好吃CP值很高',
 '花生醬給的很大方但有點太膩了歐姆蛋比較合我胃口蛋裡還包有馬鈴薯之類的可以增加飽足感',
 '鳳梨這個口味簡直人間美味來臺北絕對不能錯過的一間美式漢堡真的是很優質很好吃耶',
 '價格實惠漢堡不錯飲料還可以無限續真的佛但薯條就比較普通了',
 '雖然是美式餐廳餐點也不錯吃但應該是廚房出入口那個門開關很像大象的叫聲有如置身非洲的感覺真是太棒的體驗了再加一顆星',
 '份量是蠻大的但味道普通比較適合學生的餐廳單點了南瓜濃湯但比較像玉米湯裡面放一塊南瓜',
 '一份235元的花生醬牛肉堡花生醬很大一個就漢堡來講雖然不能算是CP值最高但這個價位對於學生還是很友善的加上還有附薯條與飲料喝到飽由於有芥末醬不是雜牌是蠻常見的醬料牌子所以我用薯條沾芥末味道中規中矩漢堡的部分要小心有些起司跟花生醬會跑出來整體吃起來肉算蠻大塊的生菜什麼的份量都不少但是給五星是有點太多不過四捨五入之後的確有五星水準飲料喝到飽的部分雖然是服務員幫你裝但是都很勤奮的問你需不需要裝就服務態度方面真的很不錯不過價格的部分可能不能太常吃聚會的話是不錯選擇',
 '漢堡好吃雞肉超嫩吃起來大大大滿足小缺點是用餐時間人很多但店內位置窄窄的',
 '建議提早訂位巔峰時段幾乎都會客滿適合朋友家庭情侶聚餐的好地方飲料可無限續杯幸福的一餐',
 '料理美味貝果漢堡肉肋排推餐點便宜服務用心',
 '菜色豐富份量蠻大的比較適合大食量的人來可以續杯價格合理是會常來的店另外很喜歡薯條炸的脆脆剛剛好',
 '點了花生醬牛肉堡雞腿堡花生的比例太高雖然拍照好看但吃一半就很膩雞腿肉不知道怎麼醃製調味的像味增又像迷迭香口味有點怪異最好吃的反而是雞塊這次消費體驗不優'

In [17]:
import jieba
#默認使用精確模式(一般直接使用精確模式即可)

#上方cut()函數中有個參數cut_all，當設定為True時是全模式，設定為False的時候是精確模式。
#精確模式不會有同的字分到兩個詞的問題，全模式則是將所有可能都列出來。
text = df.comment[32]
text = re.sub('[^\w]', '', text) #去除特殊符号（如：#、\n等）
print(text)

print('--'*20)
print('預設:', '/'.join(jieba.cut(text, cut_all=False, HMM=True))) #精確模式
print('全關閉:', '/'.join(jieba.cut(text, cut_all=False, HMM=False)))
print('全關閉:', '/'.join(jieba.cut(text, cut_all=True, HMM=True)))

cut_text = '/'.join(jieba.cut(text, cut_all=False, HMM=True))

Building prefix dict from the default dictionary ...


雞肉大戰牛肉堡超棒雞肉henjuice蘑菇牛肉堡220海陸超大盤份量很夠645
----------------------------------------


Dumping model to file cache C:\Users\howger\AppData\Local\Temp\jieba.cache
Loading model cost 0.993 seconds.
Prefix dict has been built successfully.


預設: 雞肉/大戰/牛肉/堡/超棒/雞肉/henjuice/蘑菇/牛肉/堡/220/海陸/超大/盤/份量/很夠/645
全關閉: 雞/肉/大/戰/牛肉/堡/超/棒/雞/肉/henjuice/蘑菇/牛肉/堡/220/海/陸/超大/盤/份量/很/夠/645
全關閉: 雞/肉/大/戰/牛肉/堡/超/棒/雞/肉/henjuice/蘑菇/牛肉/堡/220/海/陸/超大/盤/份量/很/夠/645


In [24]:
text = df.comment[32]
print(text)
print('---'*20)
print('/'.join(jieba.cut(text)))

雞肉大戰牛肉堡 超棒！雞肉hen juice!
蘑菇牛肉堡$220、海陸超大盤份量很夠$645！
------------------------------------------------------------
雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/
/蘑菇/牛肉/堡/$/220/、/海陸/超大/盤/份量/很夠/$/645/！


In [43]:
start_time = time.time() #計算你花了多久

clean_text_jieba = df.comment.map(lambda x:re.sub('[^\w]','',x)) # apply = map
clean_text_jieba = clean_text_jieba.apply(lambda x :'/'.join(jieba.cut(text)))
display(clean_text_jieba.to_frame())

print(f"花{time.time() - start_time}秒")

,comment
0,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...
1,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...
2,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...
3,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...
4,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...
...,...
185,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...
186,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...
187,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...
188,雞肉/大戰/牛肉/堡/ /超棒/！/雞肉/hen/ /juice/!/\n/蘑菇/牛肉/堡/...


花0.06305384635925293秒


In [70]:
pip install tensorflow

  Using cached tensorflow-2.12.0-cp311-cp311-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp311-cp311-win_amd64.whl (272.9 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.8.0-cp311-cp311-win_amd64.whl (2.6 MB)
  Using cached jax-0.4.10-py3-none-any.whl
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.54.2-cp311-cp311-win_amd64.whl (4.1 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached ml_dtypes-0.1.0-cp311-cp311-win_amd64.whl (120 kB)
  Using cached google_auth-2.18.1-py2.py3-none-any.whl (178 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached Werkzeug-2.3.4-

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [71]:
import tensorflow as tf

In [72]:
from ckiptagger import data_utils
from ckiptagger import WS, POS, NER

